In [5]:
import pandas as pd
import optuna.integration.lightgbm as lgb

/Users/takapy/.pyenv/versions/3.7.2/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning:

Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.



In [2]:
features = [
        'exc_wl',
        'distance',
        'pos_x',
        'params0',
        'params1',
        'params2',
        'params3',
        'params4',
        'params5',
        'params6',
]

In [6]:
file_path = '../data/features/'

In [7]:
X_train = pd.read_pickle(file_path + 'X_train.pkl')[features]
X_test = pd.read_pickle(file_path + 'X_test.pkl')[features]
y_train = pd.read_pickle(file_path + 'y_train.pkl')['target']

In [8]:
display(
    X_train.head(), X_train.shape,
    X_test.head(), X_test.shape,
    y_train.head(), y_train.shape
)

,exc_wl,distance,pos_x,params0,params1,params2,params3,params4,params5,params6,...,em_ev,ex_ev,ev_diff,ev_ratio,num_peak,num_strong_peak,peak_wave_length2,peak_intensity2,peak_ratio,peak_ratio2
0,1,45.607017,1313.0810,30.808589,5.811802e+02,1037.714752,1.531423,22469.651641,1032.317268,8.295610,...,1.200578,1.458824,-0.258246,0.822977,5.0,0.0,0.0,0.0,43.456948,0.0
1,2,75.894664,159.4150,91.300897,1.740582e+04,1080.510452,4.766233,33257.123175,1077.468855,8.018225,...,1.149204,1.589744,-0.440540,0.722886,4.0,1.0,0.0,0.0,25.269679,0.0
2,2,67.742158,-610.7688,106.642946,1.000000e-10,1119.464438,2.000000,42579.867913,1378.883338,11.687417,...,0.898169,1.589744,-0.691574,0.564977,4.0,1.0,0.0,0.0,15.912632,0.0
3,2,43.737855,1214.6180,306.933674,1.099486e+04,1139.855067,5.198692,39349.741703,1145.212849,9.445029,...,1.081982,1.589744,-0.507762,0.680602,3.0,1.0,0.0,0.0,6.134806,0.0
4,2,23.706539,-257.6161,46.133256,2.227622e+04,1120.918337,5.668012,31054.928673,1117.107782,7.658710,...,1.107265,1.589744,-0.482478,0.696506,6.0,1.0,0.0,0.0,28.931660,0.0


(7436, 607)

,exc_wl,distance,pos_x,params0,params1,params2,params3,params4,params5,params6,...,em_ev,ex_ev,ev_diff,ev_ratio,num_peak,num_strong_peak,peak_wave_length2,peak_intensity2,peak_ratio,peak_ratio2
7436,2,91.214034,189.6556,-0.200403,26861.147936,1377.632654,12.273408,4.999833e-13,1000.000005,44730.274402,...,0.900880,1.589744,-0.688864,0.566683,4.0,1.0,0.000,0.0,24.461780,0.000000
7437,2,94.366308,-256.4733,308.514851,26886.386343,1277.465325,9.919383,5.096237e-13,1001.383038,26753.813871,...,0.971313,1.589744,-0.618431,0.610987,4.0,1.0,0.000,0.0,8.526922,0.000000
7438,1,43.829214,-291.3372,48.078481,18270.435476,1497.343413,10.583315,1.037360e+04,1490.660842,11.663950,...,0.828712,1.458824,-0.630112,0.568069,6.0,1.0,0.000,0.0,8.822853,0.000000
7439,1,45.044423,-863.9271,179.052601,7646.843080,1498.851981,5.851480,1.899570e+04,1491.204735,13.616758,...,0.827569,1.458824,-0.631254,0.567285,3.0,2.0,1362.785,914.0,4.866763,3.095492
7440,2,32.557641,737.2772,-7414.835134,77855.568427,1172.532492,10.794408,2.827082e+07,1179.643483,2400.602334,...,1.056592,1.589744,-0.533152,0.664630,5.0,1.0,0.000,0.0,20.873492,0.000000


(6952, 607)

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

(7436,)

In [9]:
from sklearn.metrics import average_precision_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
def pr_auc(preds, data):
    y_true = data.get_label()
    score = average_precision_score(y_true, preds)
    return "pr_auc", score, True

In [10]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'gbdt',
}

In [11]:
tr_x, va_x, tr_y, va_y = train_test_split(X_train, y_train, test_size=0.3, shuffle=True, random_state=0, stratify=y_train)

In [12]:
display(
    tr_x.head(), tr_x.shape,
    va_x.head(), va_x.shape,
    tr_y.head(), tr_y.shape,
    va_y.head(), va_y.shape
)

,exc_wl,distance,pos_x,params0,params1,params2,params3,params4,params5,params6,...,em_ev,ex_ev,ev_diff,ev_ratio,num_peak,num_strong_peak,peak_wave_length2,peak_intensity2,peak_ratio,peak_ratio2
6071,1,36.674242,412.8994,70.491372,65.209318,1466.663865,0.534087,29489.588633,1445.773552,10.683801,...,0.858189,1.458824,-0.600634,0.588275,5.0,2.0,1211.895,1179.0,11.169737,6.993691
5588,1,16.763055,1471.3180,378.998337,13772.035338,1410.170178,5.172881,2880.620258,1420.450501,5.748881,...,0.879652,1.458824,-0.579172,0.602987,5.0,1.0,0.000,0.0,8.624902,0.000000
3029,2,78.587531,-287.6901,87.881073,7.114627,1151.256442,0.978902,34403.958884,1170.363511,9.373946,...,1.059737,1.589744,-0.530007,0.666609,4.0,1.0,0.000,0.0,15.387290,0.000000
5171,1,55.226805,-1086.4150,202.719271,5243.961568,1096.053205,3.730720,13501.667449,1093.925574,7.062014,...,1.131614,1.458824,-0.327210,0.775703,6.0,1.0,0.000,0.0,7.172835,0.000000
4199,2,90.005555,-961.7723,13.976079,544.681501,1164.830531,1.474276,21801.714519,1163.589804,10.054806,...,1.064493,1.589744,-0.525251,0.669600,6.0,1.0,0.000,0.0,26.492349,0.000000


(5205, 607)

,exc_wl,distance,pos_x,params0,params1,params2,params3,params4,params5,params6,...,em_ev,ex_ev,ev_diff,ev_ratio,num_peak,num_strong_peak,peak_wave_length2,peak_intensity2,peak_ratio,peak_ratio2
3830,1,92.541882,608.3837,94.662811,12510.459614,1312.715387,1.503035,3.712007e+04,1320.005654,15.952679,...,0.944394,1.458824,-0.514430,0.647367,4.0,1.0,0.0,0.0,36.066906,0.0
1350,1,51.865210,-1479.9370,41.651094,51930.786126,1276.489808,10.028332,5.005090e-13,1597.000002,51672.972747,...,0.972150,1.458824,-0.486673,0.666393,5.0,1.0,0.0,0.0,24.546740,0.0
6135,1,48.083261,478.4027,-2600.426114,29920.681609,1094.386451,13.555428,1.257228e+07,1599.999996,2418.910752,...,1.131614,1.458824,-0.327210,0.775703,5.0,1.0,0.0,0.0,5.776248,0.0
1730,1,73.437048,-674.3403,194.922771,7466.146416,1359.675777,5.798357,1.261177e+04,1355.040340,9.521707,...,0.912589,1.458824,-0.546234,0.625565,4.0,1.0,0.0,0.0,7.003423,0.0
3034,1,86.023253,-991.6271,15.195474,659.931379,1038.057799,2.874518,4.101887e+04,1085.663139,10.451509,...,1.142543,1.458824,-0.316280,0.783195,5.0,1.0,0.0,0.0,29.178450,0.0


(2231, 607)

6071    0
5588    0
3029    0
5171    0
4199    0
Name: target, dtype: int64

(5205,)

3830    1
1350    0
6135    0
1730    0
3034    0
Name: target, dtype: int64

(2231,)

In [13]:
dtrain = lgb.Dataset(tr_x, tr_y)
dvalid = lgb.Dataset(va_x, va_y)

In [14]:
%%time
best_params, tuning_history = dict(), list()
booster = lgb.train(params, dtrain,
                    valid_sets=dvalid,
                    verbose_eval=-1,
                    best_params=best_params,
                    tuning_history=tuning_history)

/Users/takapy/.pyenv/versions/3.7.2/lib/python3.7/site-packages/optuna/_experimental.py:61: ExperimentalWarning:

train is experimental (supported from v0.18.0). The interface can change in the future.

/Users/takapy/.pyenv/versions/3.7.2/lib/python3.7/site-packages/optuna/_experimental.py:83: ExperimentalWarning:

LightGBMTuner is experimental (supported from v1.5.0). The interface can change in the future.

min_data_in_leaf, val_score: 0.033530: 100%|##########| 5/5 [00:27<00:00,  5.57s/it]


CPU times: user 1h 44min 29s, sys: 2min 29s, total: 1h 46min 59s
Wall time: 9min 54s


In [15]:
best_params

{'lambda_l1': 4.244068385857982,
 'lambda_l2': 1.863308242015143e-08,
 'num_leaves': 31,
 'feature_fraction': 0.7520000000000001,
 'bagging_fraction': 0.5687112242554548,
 'bagging_freq': 2,
 'min_child_samples': 100,
 'objective': 'binary',
 'metric': 'binary_logloss',
 'boosting': 'gbdt'}

## CVバージョン

In [27]:
# LightGBM のデータセット表現にする
lgb_train = lgb.Dataset(X_train, y_train)

# データセットの分割方法
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 最適化するときの条件
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbosity': -1,
    'boosting': 'gbdt'
}

# 基本的には cv() 関数のオプションがそのまま渡せる
tuner_cv = lgb.LightGBMTunerCV(
    params, lgb_train,
    num_boost_round=1000,
    early_stopping_rounds=100,
    verbose_eval=20,
    folds=folds,
)

/Users/takapy/.pyenv/versions/3.7.2/lib/python3.7/site-packages/optuna/_experimental.py:83: ExperimentalWarning:

LightGBMTunerCV is experimental (supported from v1.5.0). The interface can change in the future.



In [28]:
%%time
# 最適なパラメータを探索する
tuner_cv.run()



  0%|          | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[20]	cv_agg's binary_logloss: 0.0460938 + 0.00706644
[40]	cv_agg's binary_logloss: 0.0441253 + 0.00992888
[60]	cv_agg's binary_logloss: 0.0488134 + 0.0137758
[80]	cv_agg's binary_logloss: 0.0553338 + 0.0173657
[100]	cv_agg's binary_logloss: 0.0633738 + 0.020199
[120]	cv_agg's binary_logloss: 0.0689102 + 0.0220824




feature_fraction, val_score: 0.042893:   0%|          | 0/7 [00:12<?, ?it/s]

feature_fraction, val_score: 0.042893:  14%|#4        | 1/7 [00:12<01:17, 12.96s/it][I 2020-06-01 23:41:03,800] Finished trial#0 with value: 0.04289337850798018 with parameters: {'feature_fraction': 0.7}. Best is trial#0 with value: 0.04289337850798018.


feature_fraction, val_score: 0.042893:  14%|#4        | 1/7 [00:13<01:17, 12.96s/it]

[20]	cv_agg's binary_logloss: 0.0447874 + 0.00783866
[40]	cv_agg's binary_logloss: 0.0438758 + 0.011457
[60]	cv_agg's binary_logloss: 0.0484738 + 0.0155016
[80]	cv_agg's binary_logloss: 0.0558221 + 0.0187449
[100]	cv_agg's binary_logloss: 0.0636209 + 0.022324
[120]	cv_agg's binary_logloss: 0.0683658 + 0.0237225




feature_fraction, val_score: 0.042374:  14%|#4        | 1/7 [00:24<01:17, 12.96s/it]

feature_fraction, val_score: 0.042374:  29%|##8       | 2/7 [00:24<01:02, 12.54s/it][I 2020-06-01 23:41:15,345] Finished trial#1 with value: 0.04237355379374536 with parameters: {'feature_fraction': 0.6}. Best is trial#1 with value: 0.04237355379374536.


feature_fraction, val_score: 0.042374:  29%|##8       | 2/7 [00:24<01:02, 12.54s/it]

...

[20]	cv_agg's binary_logloss: 0.0709554 + 0.00561432
[40]	cv_agg's binary_logloss: 0.0548737 + 0.00602859
[60]	cv_agg's binary_logloss: 0.0466278 + 0.00639949
[80]	cv_agg's binary_logloss: 0.042172 + 0.0068855
[100]	cv_agg's binary_logloss: 0.0394449 + 0.00693219
[120]	cv_agg's binary_logloss: 0.0373912 + 0.00729839
[140]	cv_agg's binary_logloss: 0.0360526 + 0.00778163
[160]	cv_agg's binary_logloss: 0.0352939 + 0.00736042
[180]	cv_agg's binary_logloss: 0.0344873 + 0.00756048
[200]	cv_agg's binary_logloss: 0.0341644 + 0.00814483
[220]	cv_agg's binary_logloss: 0.0335711 + 0.00823
[240]	cv_agg's binary_logloss: 0.0330863 + 0.0085412
[260]	cv_agg's binary_logloss: 0.0327068 + 0.00883538
[280]	cv_agg's binary_logloss: 0.03269 + 0.00913497
[300]	cv_agg's binary_logloss: 0.0327093 + 0.00929026
[320]	cv_agg's binary_logloss: 0.0327299 + 0.00950459
[340]	cv_agg's binary_logloss: 0.0327051 + 0.0097572
[360]	cv_agg's binary_logloss: 0.0329307 + 0.010317
[380]	cv_agg's binary_logloss: 0.0328537 + 



min_data_in_leaf, val_score: 0.032564:  60%|######    | 3/5 [00:14<00:07,  3.61s/it]

min_data_in_leaf, val_score: 0.032564:  80%|########  | 4/5 [00:14<00:03,  3.62s/it][I 2020-06-01 23:51:17,423] Finished trial#63 with value: 0.03261067898471357 with parameters: {'min_child_samples': 25}. Best is trial#60 with value: 0.03256426247337006.


min_data_in_leaf, val_score: 0.032564:  80%|########  | 4/5 [00:14<00:03,  3.62s/it]

[20]	cv_agg's binary_logloss: 0.0709554 + 0.00561432
[40]	cv_agg's binary_logloss: 0.0548737 + 0.00602859
[60]	cv_agg's binary_logloss: 0.0466278 + 0.00639949
[80]	cv_agg's binary_logloss: 0.042172 + 0.0068855
[100]	cv_agg's binary_logloss: 0.039468 + 0.0069202
[120]	cv_agg's binary_logloss: 0.0374321 + 0.00727884
[140]	cv_agg's binary_logloss: 0.03635 + 0.00780275
[160]	cv_agg's binary_logloss: 0.0355607 + 0.00792071
[180]	cv_agg's binary_logloss: 0.0347951 + 0.00797402
[200]	cv_agg's binary_logloss: 0.0344731 + 0.00855053
[220]	cv_agg's binary_logloss: 0.0335661 + 0.00852046
[240]	cv_agg's binary_logloss: 0.0331925 + 0.00907343
[260]	cv_agg's binary_logloss: 0.0330577 + 0.00910955
[280]	cv_agg's binary_logloss: 0.0329471 + 0.00949619
[300]	cv_agg's binary_logloss: 0.0328672 + 0.00954112
[320]	cv_agg's binary_logloss: 0.0329066 + 0.00974057
[340]	cv_agg's binary_logloss: 0.0327454 + 0.00976774
[360]	cv_agg's binary_logloss: 0.0328193 + 0.0102266
[380]	cv_agg's binary_logloss: 0.033034



min_data_in_leaf, val_score: 0.032564:  80%|########  | 4/5 [00:18<00:03,  3.62s/it]

min_data_in_leaf, val_score: 0.032564: 100%|##########| 5/5 [00:18<00:00,  3.69s/it]


CPU times: user 1h 49min 11s, sys: 2min 54s, total: 1h 52min 6s
Wall time: 10min 30s


In [29]:
print(tuner_cv.best_params)

{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'boosting': 'gbdt', 'lambda_l1': 0.0029365252139988236, 'lambda_l2': 0.010246689463847606, 'num_leaves': 2, 'feature_fraction': 0.44800000000000006, 'bagging_fraction': 0.8780768335818601, 'bagging_freq': 4, 'min_child_samples': 100}
